In [6]:
import sqlite3
conn = sqlite3.connect('bt.db')
cursor = conn.cursor()

In [7]:
cursor.execute('''CREATE TABLE data
                (
                    Num int,
                    Num1 float,
                    Num2 float
                    )''')
cursor.execute('''INSERT INTO data VALUES
               (1,3.5,7.2),
               (2, 5.1, 8.4),
               (3, 7.8, 6.9),
               (4, 9.0, 10.2),
               (5, 11.3, 4.5),
               (6, 13.7, 12.8)''')
conn.commit()

In [ ]:
# Bài 1 
cursor.execute('''
WITH chi_so AS (
    SELECT 
        COUNT(*) AS n,
        SUM(Num1) AS sum_x,
        SUM(Num2) AS sum_y,
        SUM(Num1 * Num1) AS sum_x2,
        SUM(Num2 * Num2) AS sum_y2,
        SUM(Num1 * Num2) AS sum_xy
    FROM data
)
SELECT 
    (n * sum_xy - sum_x * sum_y) /
    (SQRT(n * sum_x2 - sum_x * sum_x) * SQRT(n * sum_y2 - sum_y * sum_y)) AS r
FROM  chi_so
''')


for i in cursor:
    print(i)


(0.36331684731706154,)


In [19]:
print('Biến Num1 và Num2 có tương quan dương')

Biến Num1 và Num2 có tương quan dương


In [31]:
# Bài 2 
cursor.execute('''CREATE TABLE car_scores (
    Xe TEXT,
    Ngay INTEGER,
    Diem INTEGER
)''')
cursor.execute('''INSERT INTO car_scores VALUES 
('A', 1, 7), ('A', 2, 8), ('A', 3, 6), ('A', 4, 7),
('B', 1, 6), ('B', 2, 7), ('B', 3, 8), ('B', 4, 9),
('C', 1, 8), ('C', 2, 7), ('C', 3, 9), ('C', 4, 6);
''')

In [33]:
cursor.execute('''select * from car_scores''')
for i in cursor:
    print(i)

('A', 1, 7)
('A', 2, 8)
('A', 3, 6)
('A', 4, 7)
('B', 1, 6)
('B', 2, 7)
('B', 3, 8)
('B', 4, 9)
('C', 1, 8)
('C', 2, 7)
('C', 3, 9)
('C', 4, 6)


In [42]:
cursor.execute('''SELECT 
    Xe,
    CASE 
        WHEN Diem <= 6 THEN 'Thap'
        WHEN Diem <= 8 THEN 'Trung_binh'
        ELSE 'Cao'
    END AS PhanLoai,
    COUNT(*) AS SoLuong
FROM car_scores
GROUP BY Xe, PhanLoai
ORDER BY Xe, PhanLoai;

''')

for i in cursor:
    print(i)

('A', 'Thap', 1)
('A', 'Trung_binh', 3)
('B', 'Cao', 1)
('B', 'Thap', 1)
('B', 'Trung_binh', 2)
('C', 'Cao', 1)
('C', 'Thap', 1)
('C', 'Trung_binh', 2)


In [48]:
import pandas as pd
df = pd.read_sql_query("SELECT * FROM car_scores", conn)

In [51]:
df

,Xe,Ngay,Diem
0,A,1,7
1,A,2,8
2,A,3,6
3,A,4,7
4,B,1,6
5,B,2,7
6,B,3,8
7,B,4,9
8,C,1,8
9,C,2,7


In [52]:
from scipy.stats import chi2_contingency
def score_category(score):
    if score <= 6.5:
        return "Thấp"
    elif score <= 7.5:
        return "Trung bình"
    else:
        return "Cao"

df["Category"] = df["Diem"].apply(score_category)

# 6. Tạo bảng chéo theo Car × Category
table_car = pd.crosstab(df["Xe"], df["Category"])
chi2_car, p_car, _, _ = chi2_contingency(table_car)

# 7. Tạo bảng chéo theo Day × Category
table_day = pd.crosstab(df["Ngay"], df["Category"])
chi2_day, p_day, _, _ = chi2_contingency(table_day)

# 8. In kết quả
print("==> Kiểm định theo Car:")
print(table_car)
print(f"Chi2 = {chi2_car:.3f}, p = {p_car:.3f}")
print("=> Có sự khác biệt đáng kể giữa các mẫu xe" if p_car < 0.05 else "=> Không có sự khác biệt đáng kể")

print("\n==> Kiểm định theo Day:")
print(table_day)
print(f"Chi2 = {chi2_day:.3f}, p = {p_day:.3f}")
print("=> Kết quả phụ thuộc vào ngày" if p_day < 0.05 else "=> Kết quả không phụ thuộc vào ngày")

# 9. Đóng kết nối
conn.close()

==> Kiểm định theo Car:
Category  Cao  Thấp  Trung bình
Xe                             
A           1     1           2
B           2     1           1
C           2     1           1
Chi2 = 0.900, p = 0.925
=> Không có sự khác biệt đáng kể

==> Kiểm định theo Day:
Category  Cao  Thấp  Trung bình
Ngay                           
1           1     1           1
2           1     0           2
3           2     1           0
4           1     1           1
Chi2 = 3.600, p = 0.731
=> Kết quả không phụ thuộc vào ngày


In [ ]:
# Bài 3
conn = sqlite3.connect('flights.db')
cursor = conn.cursor()

In [60]:
# 2. Tạo bảng flights và chèn dữ liệu mẫu
cursor.execute("""
CREATE TABLE flights (
    id INTEGER PRIMARY KEY,
    departure_time INTEGER
)
""")



In [62]:
# Dữ liệu mẫu: thời gian dạng số nguyên
sample_data = [
    (1, 830),
    (2, 1445),
    (3, 30),    # 00:30 AM
    (4, 123),   # 01:23 AM
    (5, 2359),  # 23:59
    (6, 945)    # 09:45 AM
]

cursor.executemany("INSERT INTO flights (id, departure_time) VALUES (?, ?)", sample_data)
conn.commit()

# 3. Truy vấn chuyển đổi định dạng thời gian
cursor.execute('''SELECT 
    id,
    departure_time,
    printf('%02d:%02d', departure_time / 100, departure_time % 100) AS formatted_time
FROM flights''')
for i in cursor:
    print(i)

(1, 830, '08:30')
(2, 1445, '14:45')
(3, 30, '00:30')
(4, 123, '01:23')
(5, 2359, '23:59')
(6, 945, '09:45')


In [ ]:
# Bài 4
cursor.execute("CREATE TABLE data(val REAL)")

# 2. Chèn dữ liệu mẫu
values = [(10,), (12,), (13,), (12,), (11,), (100,), (12,), (13,), (11,), (10,), (200,)]
cursor.executemany("INSERT INTO data(val) VALUES (?)", values)
conn.commit()

# 3. Tính median
cursor.execute("""
SELECT val FROM data ORDER BY val
""")
vals = [row[0] for row in cursor.fetchall()]
n = len(vals)
median = (vals[n // 2] if n % 2 == 1 else (vals[n // 2 - 1] + vals[n // 2]) / 2)

# 4. Tính |xi - median|
abs_devs = [abs(v - median) for v in vals]
abs_devs.sort()
mad = (abs_devs[n // 2] if n % 2 == 1 else (abs_devs[n // 2 - 1] + abs_devs[n // 2]) / 2)

# 5. Chọn các giá trị là ngoại lệ (vượt 1.5 × MAD)
threshold = 1.5 * mad
outliers = [v for v in vals if abs(v - median) > threshold]

# 6. Xuất kết quả
print(f"Median = {median}")
print(f"MAD = {mad}")
print(f"Ngưỡng ngoại lệ (1.5×MAD): ±{threshold}")
print("Giá trị ngoại lệ:", outliers)


Median = 12.0
MAD = 1.0
Ngưỡng ngoại lệ (1.5×MAD): ±1.5
Giá trị ngoại lệ: [10.0, 10.0, 100.0, 200.0]


In [ ]:
# Bài 55
cursor.execute("""
CREATE TABLE Patient (
    id INTEGER PRIMARY KEY,
    last_name TEXT,
    weight REAL,
    height REAL
)
""")

# 2. Thêm dữ liệu mẫu
patients = [
    (1, 'Nguyen', 60, 170),
    (2, 'Nguyen', 60, 171),   # Giống last_name + weight → có thể là cùng người
    (3, 'Tran', 58, 165),
    (4, 'Nguyen', 65, 170),   # Cùng họ, khác cân nặng → khác người
]

cursor.executemany("INSERT INTO Patient VALUES (?, ?, ?, ?)", patients)
conn.commit()

# 3. Truy vấn kết hợp từng cặp và tính "khoảng cách Boolean"
cursor.execute("""
SELECT 
    p1.id AS id1,
    p2.id AS id2,
    p1.last_name = p2.last_name AS same_last_name,
    p1.weight = p2.weight AS same_weight,
    (p1.last_name = p2.last_name AND p1.weight = p2.weight) AS is_same_person
FROM Patient p1
JOIN Patient p2 ON p1.id < p2.id
""")
for i in cursor:
    print(i)

(1, 2, 1, 1, 1)
(1, 3, 0, 0, 0)
(1, 4, 1, 0, 0)
(2, 3, 0, 0, 0)
(2, 4, 1, 0, 0)
(3, 4, 0, 0, 0)


In [65]:
conn.close()